### Importing necessary modules

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.constraints import nonneg
import numpy as np
import pandas as pd
import nltk
import operator

### Dataset
Here Wikipedia Movie Plots dataset have been used.<br>
Dataset link: https://www.kaggle.com/jrobischon/wikipedia-movie-plots
* You can fit your own dataset

In [26]:
#data 
df = pd.read_csv('dataset/wiki_movie_plots_deduped.csv') #get it from Kaggle
STOP_WORDS = set(nltk.corpus.stopwords.words('english')) 
df.Plot.head()

0    A bartender is working at a saloon, serving dr...
1    The moon, painted with a smiling face hangs ov...
2    The film, just over a minute long, is composed...
3    Lasting just 61 seconds and consisting of two ...
4    The earliest known adaptation of the classic f...
Name: Plot, dtype: object

In [27]:
df.Plot[0]

"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]"

### Necessary Functions

In [28]:
#filtering and tokenizing function
def sentence_to_wordlist(sentence, filters="!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n?,।!‍.'0123456789০১২৩৪৫৬৭৮৯‘\u200c–“”…‘"):
    translate_dict = dict((c, ' ') for c in filters)
    translate_map = str.maketrans(translate_dict)
    wordlist = sentence.translate(translate_map).split()
    return list(filter(lambda x: x not in STOP_WORDS, wordlist))

#function for calculating word similarity
def cosine_similarity(u, v):

    '''Cosine similarity reflects the degree of similariy between u and v'''
    
    distance = 0.0
    
    ### START CODE HERE ###
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u,v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.sqrt(np.sum(np.power(u,2)))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.sum(np.power(v,2)))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = np.divide(dot,norm_u*norm_v)
    ### END CODE HERE ###
    
    return cosine_similarity

#determine simularity between two words
def word_similarity(word1,word2,weight,word_to_index,print_vec=False):
    e1 = weight[word_to_index[word1]]
    e2 = weight[word_to_index[word2]]
    if print_vec:
        print(word1,e1)
        print(word2,e2)
    return cosine_similarity(e1,e2)

#retruns most similiar words
def most_similiar(word,weight,word_to_index,count=5,print_vec=False):
    e1 = weight[word_to_index[word]]
    similarity = {}
    for v in vocab:
        if v is not word:
            e2 = weight[word_to_index[v]]
            similarity[v]=cosine_similarity(e1,e2)
            
    if print_vec:
        print(word,e)
        
    sorted_similarity = sorted(similarity.items(), key=operator.itemgetter(1),reverse=True)      
    return sorted_similarity[1:count+1]

#function for vocab and indexing
def word_idx_generate(text):
    '''
    --build vocabulary of unique words
    --create dictionary for 'indext to word' and word to index'
    '''
    # build vocabulary of unique words
    vocab = []
    for sent in text:
        for i in sent:
            if i not in vocab:
                vocab.append(i)
                
    word2id = {}
    id2word = {}
    for i,word in enumerate(vocab):
        word2id[word] = i
        id2word[i] = word
    return vocab,word2id,id2word

#dataset: CBOW Represantation
def cbow(text,window=4):
    '''Return dataset as one-hot encoded context(X) vs target(Y) words'''
    X = []
    Y = []
    X_context = []
    Y_target = []
    V = len(vocab)
    for sent in text:
        border = len(sent)
        end = window -1
        start = 0
        if len(sent)>=window:
            while (border>end):
                x_temp = []
                x_word = [] #op
                for i in range(start,end):
                    index = word_to_index[sent[i]]
                    temp = np.zeros((1,V))
                    temp[0][index] = 1
                    x_temp.append(temp)
                    x_word.append(sent[i]) #op
                    #print(i,sent[i],end,border) #op
                index = word_to_index[sent[end]]
                temp = np.zeros((1,V))
                temp[0][index] = 1
                Y.append(temp)
                X.append(np.sum(x_temp,axis=0)) #summping context words
                X_context.append(x_word)
                Y_target.append(sent[end]) #op
                start +=1
                end +=1
    
    return X,Y,x_context,y_target

#dataset: Skip-gram Represantation
def skip_gram(text,word_to_index,window=4):
    '''Return dataset as one-hot encoded context(X) vs target(Y) words'''
    X = []
    Y = []
    X_context = []
    Y_target = []
    V = len(vocab)
    for sent in text:
        border = len(sent)
        end = window-1
        start = 0
        if len(sent)>=window:
            while (border>end):
                index = word_to_index[sent[start]]
                temp = np.zeros((1,V))
                temp[0][index] = 1
                X.append(temp)
                y_temp = []
                y_word = []
                X_context.append(sent[start]) #op
                for i in range(start+1,end+1):
                    index = word_to_index[sent[i]]
                    temp = np.zeros((1,V))
                    temp[0][index] = 1
                    y_temp.append(temp)
                    y_word.append(sent[i])

                Y.append(np.sum(y_temp,axis=0)) #summping context words
                Y_target.append(y_word) #op
                start +=1
                end +=1
       
    X = np.array(X)
    Y = np.array(Y)

    return X,Y,X_context,Y_target

def contex_target(X_context,Y_target):
    '''prints context vs target words'''
    for i,x in enumerate(X_context):
        print(i,"Context :",x,'---> Target:',Y_target[i])

### Preparing Dataset for word vectors
We are taking first 200 plots for reducing time it takes

In [29]:
text = df[:200] #selecting first 200 entry
text = [sentence_to_wordlist(i.lower()) for i in text.Plot] #filtering and tokenizing plot text

In [52]:
text

[['bartender',
  'working',
  'saloon',
  'serving',
  'drinks',
  'customers',
  'fills',
  'stereotypically',
  'irish',
  'man',
  'bucket',
  'beer',
  'carrie',
  'nation',
  'followers',
  'burst',
  'inside',
  'assault',
  'irish',
  'man',
  'pulling',
  'hat',
  'eyes',
  'dumping',
  'beer',
  'head',
  'group',
  'begin',
  'wrecking',
  'bar',
  'smashing',
  'fixtures',
  'mirrors',
  'breaking',
  'cash',
  'register',
  'bartender',
  'sprays',
  'seltzer',
  'water',
  'nation',
  'face',
  'group',
  'policemen',
  'appear',
  'order',
  'everybody',
  'leave'],
 ['moon',
  'painted',
  'smiling',
  'face',
  'hangs',
  'park',
  'night',
  'young',
  'couple',
  'walking',
  'past',
  'fence',
  'learn',
  'railing',
  'look',
  'moon',
  'smiles',
  'embrace',
  'moon',
  'smile',
  'gets',
  'bigger',
  'sit',
  'bench',
  'tree',
  'moon',
  'view',
  'blocked',
  'causing',
  'frown',
  'last',
  'scene',
  'man',
  'fans',
  'woman',
  'hat',
  'moon',
  'left',

In [30]:
vocab, word2idx,idx2word = word_idx_generate(text)

In [31]:
X,Y,X_context,Y_target = skip_gram(text,word2idx)
print('X:',X.shape,'Y:',Y.shape)

X: (18360, 1, 5779) Y: (18360, 1, 5779)


Here we have followed skip-gram model. Shape of X and Y shows, we have **18360 context word against 18360 target word sets.** <br>
Here we can visualize **context word** Vs **target word** with *context_target()* function.

In [32]:
contex_target(X_context,Y_target)

0 Context : bartender ---> Target: ['working', 'saloon', 'serving']
1 Context : working ---> Target: ['saloon', 'serving', 'drinks']
2 Context : saloon ---> Target: ['serving', 'drinks', 'customers']
3 Context : serving ---> Target: ['drinks', 'customers', 'fills']
4 Context : drinks ---> Target: ['customers', 'fills', 'stereotypically']
5 Context : customers ---> Target: ['fills', 'stereotypically', 'irish']
6 Context : fills ---> Target: ['stereotypically', 'irish', 'man']
7 Context : stereotypically ---> Target: ['irish', 'man', 'bucket']
8 Context : irish ---> Target: ['man', 'bucket', 'beer']
9 Context : man ---> Target: ['bucket', 'beer', 'carrie']
10 Context : bucket ---> Target: ['beer', 'carrie', 'nation']
11 Context : beer ---> Target: ['carrie', 'nation', 'followers']
12 Context : carrie ---> Target: ['nation', 'followers', 'burst']
13 Context : nation ---> Target: ['followers', 'burst', 'inside']
14 Context : followers ---> Target: ['burst', 'inside', 'assault']
15 Context 

277 Context : large ---> Target: ['white', 'rabbit', 'rabbit']
278 Context : white ---> Target: ['rabbit', 'rabbit', 'hole']
279 Context : rabbit ---> Target: ['rabbit', 'hole', 'finds']
280 Context : rabbit ---> Target: ['hole', 'finds', 'tiny']
281 Context : hole ---> Target: ['finds', 'tiny', 'door']
282 Context : finds ---> Target: ['tiny', 'door', 'finds']
283 Context : tiny ---> Target: ['door', 'finds', 'bottle']
284 Context : door ---> Target: ['finds', 'bottle', 'labeled']
285 Context : finds ---> Target: ['bottle', 'labeled', 'drink']
286 Context : bottle ---> Target: ['labeled', 'drink', 'shrinks']
287 Context : labeled ---> Target: ['drink', 'shrinks', 'enough']
288 Context : drink ---> Target: ['shrinks', 'enough', 'pass']
289 Context : shrinks ---> Target: ['enough', 'pass', 'door']
290 Context : enough ---> Target: ['pass', 'door', 'eats']
291 Context : pass ---> Target: ['door', 'eats', 'something']
292 Context : door ---> Target: ['eats', 'something', 'labeled']
293 Co

477 Context : restores ---> Target: ['consciousness', 'dousing', 'water']
478 Context : consciousness ---> Target: ['dousing', 'water', 'comic']
479 Context : dousing ---> Target: ['water', 'comic', 'relief']
480 Context : water ---> Target: ['comic', 'relief', 'dance']
481 Context : comic ---> Target: ['relief', 'dance', 'hall']
482 Context : relief ---> Target: ['dance', 'hall', 'eastern']
483 Context : dance ---> Target: ['hall', 'eastern', 'stranger']
484 Context : hall ---> Target: ['eastern', 'stranger', 'forced']
485 Context : eastern ---> Target: ['stranger', 'forced', 'dance']
486 Context : stranger ---> Target: ['forced', 'dance', 'locals']
487 Context : forced ---> Target: ['dance', 'locals', 'fire']
488 Context : dance ---> Target: ['locals', 'fire', 'feet']
489 Context : locals ---> Target: ['fire', 'feet', 'door']
490 Context : fire ---> Target: ['feet', 'door', 'suddenly']
491 Context : feet ---> Target: ['door', 'suddenly', 'opens']
492 Context : door ---> Target: ['sud

876 Context : fight ---> Target: ['santa', 'claus', 'leaves']
877 Context : santa ---> Target: ['claus', 'leaves', 'home']
878 Context : claus ---> Target: ['leaves', 'home', 'sleigh']
879 Context : leaves ---> Target: ['home', 'sleigh', 'reindeer']
880 Context : home ---> Target: ['sleigh', 'reindeer', 'enters']
881 Context : sleigh ---> Target: ['reindeer', 'enters', 'children']
882 Context : reindeer ---> Target: ['enters', 'children', 'house']
883 Context : enters ---> Target: ['children', 'house', 'chimney']
884 Context : children ---> Target: ['house', 'chimney', 'leaves']
885 Context : house ---> Target: ['chimney', 'leaves', 'presents']
886 Context : chimney ---> Target: ['leaves', 'presents', 'children']
887 Context : leaves ---> Target: ['presents', 'children', 'come']
888 Context : presents ---> Target: ['children', 'come', 'stairs']
889 Context : children ---> Target: ['come', 'stairs', 'enjoy']
890 Context : come ---> Target: ['stairs', 'enjoy', 'presents']
891 Context : r

1276 Context : fire ---> Target: ['thug', 'rescuer', 'fight']
1277 Context : thug ---> Target: ['rescuer', 'fight', 'roof']
1278 Context : rescuer ---> Target: ['fight', 'roof', 'house']
1279 Context : young ---> Target: ['couple', 'decides', 'elope']
1280 Context : couple ---> Target: ['decides', 'elope', 'caught']
1281 Context : decides ---> Target: ['elope', 'caught', 'midst']
1282 Context : elope ---> Target: ['caught', 'midst', 'romantic']
1283 Context : caught ---> Target: ['midst', 'romantic', 'moment']
1284 Context : midst ---> Target: ['romantic', 'moment', 'woman']
1285 Context : romantic ---> Target: ['moment', 'woman', 'angry']
1286 Context : moment ---> Target: ['woman', 'angry', 'father']
1287 Context : woman ---> Target: ['angry', 'father', 'make']
1288 Context : angry ---> Target: ['father', 'make', 'plans']
1289 Context : father ---> Target: ['make', 'plans', 'leave']
1290 Context : make ---> Target: ['plans', 'leave', 'thief']
1291 Context : plans ---> Target: ['leave

...

16672 Context : tale ---> Target: ['forest', 'conditions', 'worsens']
16673 Context : forest ---> Target: ['conditions', 'worsens', 'death']
16674 Context : conditions ---> Target: ['worsens', 'death', 'tries']
16675 Context : worsens ---> Target: ['death', 'tries', 'lure']
16676 Context : death ---> Target: ['tries', 'lure', 'eternal']
16677 Context : tries ---> Target: ['lure', 'eternal', 'rest']
16678 Context : lure ---> Target: ['eternal', 'rest', 'life']
16679 Context : eternal ---> Target: ['rest', 'life', 'also']
16680 Context : rest ---> Target: ['life', 'also', 'appears']
16681 Context : life ---> Target: ['also', 'appears', 'finally']
16682 Context : also ---> Target: ['appears', 'finally', 'wins']
16683 Context : described ---> Target: ['film', 'magazine', 'alexis']
16684 Context : film ---> Target: ['magazine', 'alexis', 'caesar']
16685 Context : magazine ---> Target: ['alexis', 'caesar', 'napoleon']
16686 Context : alexis ---> Target: ['caesar', 'napoleon', 'brown']
16687 

17072 Context : house ---> Target: ['time', 'unintentionally', 'stop']
17073 Context : time ---> Target: ['unintentionally', 'stop', 'fleeing']
17074 Context : unintentionally ---> Target: ['stop', 'fleeing', 'thieves']
17075 Context : stop ---> Target: ['fleeing', 'thieves', 'bumping']
17076 Context : fleeing ---> Target: ['thieves', 'bumping', 'mr']
17077 Context : thieves ---> Target: ['bumping', 'mr', 'rough']
17078 Context : bumping ---> Target: ['mr', 'rough', 'takes']
17079 Context : mr ---> Target: ['rough', 'takes', 'back']
17080 Context : rough ---> Target: ['takes', 'back', 'necklace']
17081 Context : takes ---> Target: ['back', 'necklace', 'thieves']
17082 Context : back ---> Target: ['necklace', 'thieves', 'taken']
17083 Context : necklace ---> Target: ['thieves', 'taken', 'jail']
17084 Context : described ---> Target: ['film', 'magazine', 'review']
17085 Context : film ---> Target: ['magazine', 'review', 'sebastian']
17086 Context : magazine ---> Target: ['review', 'sebas

17472 Context : alvarado ---> Target: ['comes', 'wooing', 'last']
17473 Context : comes ---> Target: ['wooing', 'last', 'aztecs']
17474 Context : wooing ---> Target: ['last', 'aztecs', 'wins']
17475 Context : described ---> Target: ['film', 'magazine', 'susan']
17476 Context : film ---> Target: ['magazine', 'susan', 'sweeney']
17477 Context : magazine ---> Target: ['susan', 'sweeney', 'marsh']
17478 Context : susan ---> Target: ['sweeney', 'marsh', 'employed']
17479 Context : sweeney ---> Target: ['marsh', 'employed', 'doll']
17480 Context : marsh ---> Target: ['employed', 'doll', 'factory']
17481 Context : employed ---> Target: ['doll', 'factory', 'learns']
17482 Context : doll ---> Target: ['factory', 'learns', 'inherited']
17483 Context : factory ---> Target: ['learns', 'inherited', 'hotel']
17484 Context : learns ---> Target: ['inherited', 'hotel', 'small']
17485 Context : inherited ---> Target: ['hotel', 'small', 'town']
17486 Context : hotel ---> Target: ['small', 'town', 'adiron

17771 Context : affair ---> Target: ['denton', 'follows', 'estrella']
17772 Context : denton ---> Target: ['follows', 'estrella', 'room']
17773 Context : follows ---> Target: ['estrella', 'room', 'benham']
17774 Context : estrella ---> Target: ['room', 'benham', 'catches']
17775 Context : room ---> Target: ['benham', 'catches', 'misunderstands']
17776 Context : benham ---> Target: ['catches', 'misunderstands', 'sees']
17777 Context : catches ---> Target: ['misunderstands', 'sees', 'denton']
17778 Context : misunderstands ---> Target: ['sees', 'denton', 'honorably']
17779 Context : sees ---> Target: ['denton', 'honorably', 'keeps']
17780 Context : denton ---> Target: ['honorably', 'keeps', 'estrella']
17781 Context : honorably ---> Target: ['keeps', 'estrella', 'secret']
17782 Context : keeps ---> Target: ['estrella', 'secret', 'consequence']
17783 Context : estrella ---> Target: ['secret', 'consequence', 'must']
17784 Context : secret ---> Target: ['consequence', 'must', 'resign']
1778

18171 Context : berlingot ---> Target: ['transforming', 'true', 'beautiful']
18172 Context : transforming ---> Target: ['true', 'beautiful', 'appearance']
18173 Context : true ---> Target: ['beautiful', 'appearance', 'insists']
18174 Context : beautiful ---> Target: ['appearance', 'insists', 'children']
18175 Context : appearance ---> Target: ['insists', 'children', 'search']
18176 Context : insists ---> Target: ['children', 'search', 'bluebird']
18177 Context : children ---> Target: ['search', 'bluebird', 'happiness']
18178 Context : search ---> Target: ['bluebird', 'happiness', 'gives']
18179 Context : bluebird ---> Target: ['happiness', 'gives', 'tyltyl']
18180 Context : happiness ---> Target: ['gives', 'tyltyl', 'magical']
18181 Context : gives ---> Target: ['tyltyl', 'magical', 'hat']
18182 Context : tyltyl ---> Target: ['magical', 'hat', 'power']
18183 Context : magical ---> Target: ['hat', 'power', 'show']
18184 Context : hat ---> Target: ['power', 'show', 'insides']
18185 Conte

In [33]:
#reshaping dataset to fit in neural network
r = [X.shape[0],X.shape[2]]
X = X.reshape(r)
Y = Y.reshape(r)

### Model Definition

In [34]:
# create model
model = Sequential()
model.add(Dense(100, input_dim=r[1], init= 'uniform' ,kernel_constraint=nonneg(), activation= 'sigmoid' ))
model.add(Dense(r[1], init= 'uniform',kernel_constraint=nonneg(), activation= 'softmax' ))

model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])

W0829 21:32:45.312567  9500 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, input_dim=5779, kernel_constraint=<keras.con..., activation="sigmoid", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
W0829 21:32:46.774575  9500 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 21:32:46.972735  9500 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.rando

### Training Word Vectors

In [35]:
train = model.fit(X, Y, nb_epoch=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
W0829 21:32:48.037503  9500 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
18360/18360 [==============================] - 33s 2ms/step - loss: 0.0044 - acc: 0.9995


### Word Similarity Analysis
Now we will use the weight matrix of our Neural Network.<br>
We have two weight metrices for two layers. We have taken 100 hidden units in hidden layer. So, for each word we have 100 dimentional vector.
<br><br>
Weights from both layers need to be summed for final vector.
<br><br>
**Here is an important thing:** Word vectors have less dimention than the one-hot encoding matrix. It reduce computation complexity in NLP task. 

In [36]:
weight1, biases = model.layers[0].get_weights()
weight2, biases = model.layers[1].get_weights()
weight = weight1 +weight2.T

In [37]:
print(word_similarity('afternoon','dead',weight,word2idx))

0.7319193


In [38]:
print(most_similiar('man',weight,word2idx))

[('wife', 0.95879227), ('find', 0.9579319), ('woman', 0.9579221), ('girl', 0.9572276), ('young', 0.9562159)]


### Reproducing Past Results 
In this section weights from the first training article to reproduce the past result that has been discussed in the medium article

In [39]:
from sklearn.externals import joblib

In [40]:
filename = 'model.sav'
#joblib.dump(model, filename)

In [41]:
loaded_model = joblib.load(filename)

In [42]:
W1, biases = loaded_model.layers[0].get_weights()
W2, biases = loaded_model.layers[1].get_weights()
W = W1 +W2.T

In [43]:
print(most_similiar('man',W,word2idx))

[('father', 0.95607513), ('young', 0.9553033), ('wife', 0.94956106), ('back', 0.94942164), ('comes', 0.9466744)]


In [44]:
W[0]

array([0.00821561, 0.04129512, 0.00794185, 0.00780243, 0.01642974,
       0.03365516, 0.01118944, 0.03263743, 0.02428928, 0.0077619 ,
       0.00802749, 0.01934635, 0.03983404, 0.00800348, 0.06027178,
       0.03288328, 0.04176749, 0.01901086, 0.017279  , 0.02211751,
       0.05443039, 0.01153891, 0.00776618, 0.06963662, 0.00784243,
       0.00779849, 0.06005738, 0.07359082, 0.03126929, 0.0289063 ,
       0.01402762, 0.06246269, 0.01757432, 0.0672893 , 0.03275475,
       0.02430024, 0.04265555, 0.04689769, 0.01273581, 0.0290504 ,
       0.02346605, 0.00780975, 0.00773496, 0.04264919, 0.01187569,
       0.047987  , 0.06579044, 0.03375261, 0.00777987, 0.02008096,
       0.02719541, 0.0379695 , 0.0289874 , 0.00775713, 0.00772526,
       0.00790034, 0.01972924, 0.03903933, 0.00775233, 0.04250645,
       0.03706887, 0.00781126, 0.00775632, 0.03585426, 0.02283861,
       0.07334395, 0.00779458, 0.01583624, 0.0104772 , 0.0528849 ,
       0.0079683 , 0.0080232 , 0.00776479, 0.05910534, 0.00817

In [45]:
W

array([[ 0.00821561,  0.04129512,  0.00794185, ...,  0.02667897,
         0.00781133,  0.0078567 ],
       [ 0.01905388,  0.04548687,  0.08296685, ...,  0.06120104,
         0.08697856,  0.04136325],
       [ 0.07770883,  0.08835563,  0.04529092, ...,  0.0320814 ,
         0.06857783,  0.04764068],
       ...,
       [ 0.00774618,  0.04764719,  0.00251844, ...,  0.01819674,
         0.07319212,  0.00442693],
       [ 0.03006158,  0.035986  ,  0.01733726, ..., -0.        ,
         0.00018429,  0.01144186],
       [ 0.01860016,  0.02935425,  0.01771664, ...,  0.00601502,
         0.03184192,  0.04288519]], dtype=float32)

In [46]:
len(list((word2idx)))

5779

In [47]:
W[0][0]

0.008215612

In [48]:
X.shape

(18360, 5779)

In [49]:
word2idx

{'bartender': 0,
 'working': 1,
 'saloon': 2,
 'serving': 3,
 'drinks': 4,
 'customers': 5,
 'fills': 6,
 'stereotypically': 7,
 'irish': 8,
 'man': 9,
 'bucket': 10,
 'beer': 11,
 'carrie': 12,
 'nation': 13,
 'followers': 14,
 'burst': 15,
 'inside': 16,
 'assault': 17,
 'pulling': 18,
 'hat': 19,
 'eyes': 20,
 'dumping': 21,
 'head': 22,
 'group': 23,
 'begin': 24,
 'wrecking': 25,
 'bar': 26,
 'smashing': 27,
 'fixtures': 28,
 'mirrors': 29,
 'breaking': 30,
 'cash': 31,
 'register': 32,
 'sprays': 33,
 'seltzer': 34,
 'water': 35,
 'face': 36,
 'policemen': 37,
 'appear': 38,
 'order': 39,
 'everybody': 40,
 'leave': 41,
 'moon': 42,
 'painted': 43,
 'smiling': 44,
 'hangs': 45,
 'park': 46,
 'night': 47,
 'young': 48,
 'couple': 49,
 'walking': 50,
 'past': 51,
 'fence': 52,
 'learn': 53,
 'railing': 54,
 'look': 55,
 'smiles': 56,
 'embrace': 57,
 'smile': 58,
 'gets': 59,
 'bigger': 60,
 'sit': 61,
 'bench': 62,
 'tree': 63,
 'view': 64,
 'blocked': 65,
 'causing': 66,
 'frown'

In [50]:
X


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
X_context

['bartender',
 'working',
 'saloon',
 'serving',
 'drinks',
 'customers',
 'fills',
 'stereotypically',
 'irish',
 'man',
 'bucket',
 'beer',
 'carrie',
 'nation',
 'followers',
 'burst',
 'inside',
 'assault',
 'irish',
 'man',
 'pulling',
 'hat',
 'eyes',
 'dumping',
 'beer',
 'head',
 'group',
 'begin',
 'wrecking',
 'bar',
 'smashing',
 'fixtures',
 'mirrors',
 'breaking',
 'cash',
 'register',
 'bartender',
 'sprays',
 'seltzer',
 'water',
 'nation',
 'face',
 'group',
 'policemen',
 'appear',
 'moon',
 'painted',
 'smiling',
 'face',
 'hangs',
 'park',
 'night',
 'young',
 'couple',
 'walking',
 'past',
 'fence',
 'learn',
 'railing',
 'look',
 'moon',
 'smiles',
 'embrace',
 'moon',
 'smile',
 'gets',
 'bigger',
 'sit',
 'bench',
 'tree',
 'moon',
 'view',
 'blocked',
 'causing',
 'frown',
 'last',
 'scene',
 'man',
 'fans',
 'woman',
 'hat',
 'moon',
 'left',
 'sky',
 'perched',
 'shoulder',
 'film',
 'minute',
 'long',
 'composed',
 'two',
 'shots',
 'first',
 'girl',
 'sits',